## Computational Challenge 4

Group \#2

Students: Thuyen Dang, Chunmei Sun, Jayson Cortez, and Alan Akil

Course: Math 6397 - Stochastic Processes in Biology

Professors: Dr. Josić & Dr. Stewart

$\textbf{Challenge:}$


In this challenge you will implement stochastic simulations of two synthetic biological circuits. The synthetic genetic switch, and the repressilator were described in two back–to–back papers in Nature in 2000. Please follow the link on the website to the two papers.
In this challenge two groups will report on the deterministic version of models of these circuits, while the other two will report on the stochastic versions. The last part of each question asks that you compare the results of the simulations. Therefore groups 1 and 2, and groups 3 and 4 will need to work together on each project.


Group 2:
Consider the exclusive switch discussed in section III of the paper by Loinger, et al. (the one discussing the stochastic genetic switch). The master equation for this system is given in Eq. (14)
- Construct a Petri net representation of the system based on the description in the text and the master equation.
- Set up a stochastic simulation of this system. Use the parameters discussed in the text, and change $k$ from 0.005 to 50, as in figure 4. What happens to the switching times between the two states as $k$ is increased? You can use the Gillespie algorithm. Show the traces for $N_A$ and $N_B$ in three or four cases. Discuss whether the change is as expected.
- Together with Group 1 explain whether or whether not these results can be explained using the deterministic equations.

## Petri net representation of the system

![title](./Petrinet_comp_challenge_4.jpg)

In [3]:
# Import packages needed.
import numpy as np
import random
from scipy.special import gamma, factorial
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import tqdm
from numba import jit
import time
from scipy.integrate import solve_ivp

In [2]:
def ssa(x,g,d,a_0,a_1,T):
    # x=[A,rA,B,rB] is the molecule count of each species.
    t_x=[0] # reaction time of fully-observed trajectory
    i=0
    # Reaction matrix.
    v=np.array([[1,0,0,0],[-1,0,0,0],[-1,1,0,0],[1,-1,0,0],[0,0,1,0],[0,0,-1,0],[0,0,-1,1],[0,0,1,-1]])
    while t_x[i]<T:
        # Propensity functions
        a = np.array([g*(1-x[i,3]),d*x[i,0],a_0*x[i,0]*(1-x[i,1]-x[i,3]),a_1*x[i,1],g*(1-x[i,1]),
                      d*x[i,2],a_0*x[i,2]*(1-x[i,1]-x[i,3]),a_1*x[i,3]])
        sum_a=np.sum(a)
        cumsum_a=np.cumsum(a,axis=0)
        # Draw 2 random numbers from U(0,1)
        epsilon=np.random.uniform(0,1,2)
        # Find the reaction that will take place next.
        j=np.min(np.where(epsilon[0]*sum_a<cumsum_a))
        # Compute the time until reaction j happens.
        tau=(np.log(1/epsilon[1]))/sum_a
        # Update molecule counts due to reaction j happening.
        x=np.append(x,[x[i,:]+v[j,:]],axis=0)
        # Update time of simulation.
        t_x=np.append(t_x,[t_x[i]+tau])
        i+=1
    return x,t_x